In [ ]:
#| default_exp core

# dialoghelper

In [ ]:
#| export
import json, importlib, linecache
from typing import Dict
from tempfile import TemporaryDirectory
from ipykernel_helper import *
from dataclasses import dataclass
from fastcore.xml import to_xml

from fastcore.utils import *
from fastcore.meta import delegates
from ghapi.all import *
from fastlite import *
from fastcore.xtras import asdict
from inspect import currentframe,Parameter,signature
from httpx import get as xget, post as xpost
from dialoghelper.core import __all__ as _all
from IPython.display import display,Markdown

Re-export asdict:

In [ ]:
#| export
_all_ = ["asdict"]

In [ ]:
#| export
def find_var(var:str):
    "Search for var in all frames of the call stack"
    frame = currentframe()
    while frame:
        dv = frame.f_globals.get(var, frame.f_locals.get(var, None))
        if dv: return dv
        frame = frame.f_back
    raise ValueError(f"Could not find {var} in any scope")

In [ ]:
a = 1
find_var('a')

1

In [ ]:
#| export
def call_endp(path, dname='', json=False, raiseex=False, **data):
    if not dname: dname = find_dname()
    data['dlg_name'] = dname
    res = xpost(f'http://localhost:5001/{path}', data=data)
    if raiseex: res.raise_for_status()
    try: return res.json() if json else res.text
    except Exception as e: return str(e)

In [ ]:
#| export
def find_dname():
    "Get the message id by searching the call stack for __dialog_id."
    return find_var('__dialog_name')

In [ ]:
#| export
def find_msg_id():
    "Get the message id by searching the call stack for __dialog_id."
    return find_var('__msg_id')

In [ ]:
__msg_id = found[0]['id']

In [ ]:
find_msg_id()

'_7e1900cd'

In [ ]:
#| export
def curr_dialog(
    with_messages:bool=False,  # Include messages as well?
    dname:str='' # Running dialog to get info for; defaults to current dialog
):
    "Get the current dialog info."
    res = call_endp('curr_dialog_', dname, json=True, with_messages=with_messages)
    if res: return {'name': res['name'], 'mode': res['mode']}

In [ ]:
#| export
def find_msgs(
    re_pattern:str='', # Optional regex to search for (re.DOTALL+re.MULTILINE is used)
    msg_type:str=None, # optional limit by message type ('code', 'note', or 'prompt')
    limit:int=None, # Optionally limit number of returned items
    include_output:bool=True, # Include output in returned dict?
    dname:str='' # Running dialog to get info for; defaults to current dialog
):
    "Find `list[dict]` of messages in current specific dialog that contain the given information. To refer to a message found later, use its `id` field."
    res = call_endp('find_msgs_', dname, json=True, re_pattern=re_pattern, msg_type=msg_type, limit=limit)['msgs']
    if not include_output:
        for o in res: o.pop('output', None)
    return res

In [ ]:
# NB: must have a dialogue open including a message with this text in its content
txt = 'tools'
found = find_msgs(txt)
found[0]['content']

'Available tools: &`[add,mult,weather,username]`. Use only where required or requested.'

In [ ]:
#| export
def msg_idx(
    msgid=None,  # Message id to find (defaults to current message)
    dname:str='' # Running dialog to get info for; defaults to current dialog
):
    "Get absolute index of message in dialog."
    if not msgid: msgid = find_msg_id()
    return call_endp('msg_idx_', dname, json=True, msgid=msgid)['msgid']

In [ ]:
msg_idx()

2

In [ ]:
#| export
def read_msg(
    n:int=-1,     # Message index (if relative, +ve is downwards)
    msgid=None,  # Message id to find (defaults to current message)
    relative:bool=True,  # Is `n` relative to current message (True) or absolute (False)?
    dname:str='' # Running dialog to get info for; defaults to current dialog
    ):
    "Get the `Message` object indexed in the current dialog."
    if not msgid: msgid = find_msg_id()
    return call_endp('read_msg_', dname, json=True, msgid=msgid, n=n, relative=relative)['msg']

In [ ]:
# Previous message relative to current
read_msg(-1)['content']

'from solveit.core import rt,app\nfrom fasthtml.common import *\n\ndef add(x:float, y:float):\n    "Add x and y"\n    return x+y\n\ndef mult(x:float, y:float):\n    "Multiply x and y"\n    return x*y\n\ndef weather(city:str):\n    "Get weather for city"\n    return f"Sunny and clear"\n\ndef username():\n    "Get username"\n    return "jph00"\n'

In [ ]:
# Last message in dialog
read_msg(-1, relative=False)['content']

'import dialoghelper'

In [ ]:
#| export
def add_html(
    content:str, # The HTML to send to the client (generally should include hx-swap-oob)
    dname:str='' # Running dialog to get info for; defaults to current dialog
):
    "Send HTML to the browser to be swapped into the DOM"
    call_endp('add_html_', dname, content=to_xml(content))

In [ ]:
from fasthtml.common import *

In [ ]:
add_html(Div(P('Hi'), hx_swap_oob='beforeend:#dialog-container'))

In [ ]:
#| export
def del_msg(
    msgid:str=None, # id of message to delete
    dname:str='' # Running dialog to get info for; defaults to current dialog
):
    "Delete a message from the dialog."
    call_endp('rm_msg_', dname, raiseex=True, msid=msgid)

In [ ]:
# msg = find_msgs('aaaaa')[0]
# _id = msg['id']
# _id

In [ ]:
# del_msg(_id)

In [ ]:
#| export
def run_msg(
    msgid:str=None, # id of message to execute
    dname:str='' # Running dialog to get info for; defaults to current dialog
):
    "Adds a message to the run queue. Use read_msg to see the output once it runs."
    return call_endp('add_runq_', dname, msgid=msgid, api=True)

In [ ]:
run_msg('_312e03f5')

'{"status":"queued"}'

In [ ]:
#| export
Placements = str_enum('Placements', 'add_after', 'add_before', 'at_start', 'at_end')

In [ ]:
#| export
def add_msg(
    content:str, # Content of the message (i.e the message prompt, code, or note text)
    placement:str='add_after', # Can be 'add_after', 'add_before', 'at_start', 'at_end'
    msgid:str=None, # id of message that placement is relative to (if None, uses current message)
    msg_type: str='note', # Message type, can be 'code', 'note', or 'prompt'
    output:str='', # For prompts/code, initial output
    time_run: str | None = '', # When was message executed
    is_exported: int | None = 0, # Export message to a module?
    skipped: int | None = 0, # Hide message from prompt?
    i_collapsed: int | None = 0, # Collapse input?
    o_collapsed: int | None = 0, # Collapse output?
    heading_collapsed: int | None = 0, # Collapse heading section?
    pinned: int | None = 0, # Pin to context?
    dname:str='' # Running dialog to get info for; defaults to current dialog
):
    "Add/update a message to the queue to show after code execution completes."
    if placement not in ('at_start','at_end') and not msgid: msgid = find_msg_id()
    return call_endp(
        'add_relative_', dname, content=content, placement=placement, msgid=msgid, msg_type=msg_type, output=output,
        time_run=time_run, is_exported=is_exported, skipped=skipped, pinned=pinned,
        i_collapsed=i_collapsed, o_collapsed=o_collapsed, heading_collapsed=heading_collapsed)

In [ ]:
_id = add_msg('testing')

In [ ]:
del_msg(_id)

In [ ]:
#| export
@delegates(add_msg)
def _add_msg_unsafe(
    content:str, # Content of the message (i.e the message prompt, code, or note text)
    placement:str='add_after', # Can be 'add_after', 'add_before', 'at_start', 'at_end'
    msgid:str=None, # id of message that placement is relative to (if None, uses current message)
    run:bool=False, # For prompts, send it to the AI; for code, execute it (*DANGEROUS -- be careful of what you run!)
    **kwargs
):
    """Add/update a message to the queue to show after code execution completes, and optionally run it. Be sure to pass a `sid` (stable id) not a `mid` (which is used only for sorting, and can change).
    *WARNING*--This can execute arbitrary code, so check carefully what you run!--*WARNING"""
    if placement not in ('at_start','at_end') and not msgid: msgid = find_msg_id()
    dname = kwargs.pop('dname')
    return call_endp(
        'add_relative_', dname, content=content, placement=placement, msgid=msgid, run=run, **kwargs)

In [ ]:
_id = _add_msg_unsafe('1+1', run=True, msg_type='code')

In [ ]:
del_msg(_id)

In [ ]:
_id = _add_msg_unsafe('Hi', run=True, msg_type='prompt')

In [ ]:
del_msg(_id)

In [ ]:
#| export
def _umsg(
    content:str|None=None, # Content of the message (i.e the message prompt, code, or note text)
    msg_type: str|None = None, # Message type, can be 'code', 'note', or 'prompt'
    output:str|None = None, # For prompts/code, the output
    time_run: str | None = None, # When was message executed
    is_exported: int | None = None, # Export message to a module?
    skipped: int | None = None, # Hide message from prompt?
    i_collapsed: int | None = None, # Collapse input?
    o_collapsed: int | None = None, # Collapse output?
    heading_collapsed: int | None = None, # Collapse heading section?
    pinned: int | None = None # Pin to context?
): ...

In [ ]:
#| export
@delegates(_umsg)
def update_msg(
    msgid:str=None, # id of message to update (if None, uses current message)
    msg:Optional[Dict]=None, # Dictionary of field keys/values to update
    dname:str='', # Running dialog to get info for; defaults to current dialog
    **kwargs):
    """Update an existing message. Provide either `msg` OR field key/values to update.
    Use `content` param to update contents.
    Only include parameters to update--missing ones will be left unchanged."""
    if not msgid and not msg: raise TypeError("update_msg needs either a dict message or `msgid=`")
    return call_endp('add_relative_', dname, placement='update', msgid=msgid, **kwargs)

In [ ]:
_id = add_msg('testing')
_id

'_2d24071a'

In [ ]:
update_msg(_id, content='toasting')

'_2d24071a'

In [ ]:
update_msg(_id, skipped=1)

'_2d24071a'

In [ ]:
_id

'_2d24071a'

In [ ]:
del_msg(_id)

## Gists

In [ ]:
#| export
def load_gist(gist_id:str):
    "Retrieve a gist"
    api = GhApi()
    if '/' in gist_id: *_,user,gist_id = gist_id.split('/')
    else: user = None
    return api.gists.get(gist_id, user=user)

In [ ]:
gistid = 'jph00/e7cfd4ded593e8ef6217e78a0131960c'
gist = load_gist(gistid)
gist.html_url

'https://gist.github.com/jph00/e7cfd4ded593e8ef6217e78a0131960c'

In [ ]:
#| export
def gist_file(gist_id:str):
    "Get the first file from a gist"
    gist = load_gist(gist_id)
    return first(gist.files.values())

In [ ]:
gfile = gist_file(gistid)
print(gfile.content[:100]+"…")

"This is a test module which makes some simple tools available."
__all__ = ["hi","whoami"]

testfoo=…


In [ ]:
#| export
def import_string(
    code:str, # Code to import as a module
    name:str  # Name of module to create
):
    with TemporaryDirectory() as tmpdir:
        path = Path(tmpdir) / f"{name}.py"
        path.write_text(code)
        # linecache.cache storage allows inspect.getsource() after tmpdir lifetime ends
        linecache.cache[str(path)] = (len(code), None, code.splitlines(keepends=True), str(path))
        spec = importlib.util.spec_from_file_location(name, path)
        module = importlib.util.module_from_spec(spec)
        sys.modules[name] = module
        spec.loader.exec_module(module)
        return module

In [ ]:
#| export
empty = Parameter.empty

def is_usable_tool(func:callable):
    "True if the function has a docstring and all parameters have types, meaning that it can be used as an LLM tool."    
    if not func.__doc__ or not callable(func): return False
    return all(p.annotation != empty for p in signature(func).parameters.values())

In [ ]:
def hi(who:str):
    "Say hi to `who`"
    return f"Hello {who}"

def hi2(who):
    "Say hi to `who`"
    return f"Hello {who}"

def hi3(who:str):
    return f"Hello {who}"

bye = "bye"

In [ ]:
assert is_usable_tool(hi)
assert not is_usable_tool(hi2)
assert not is_usable_tool(hi3)
assert not is_usable_tool(bye)

In [ ]:
#| export
def mk_toollist(syms):
    return "\n".join(f"- &`{sym.__name__}`: {sym.__doc__}" for sym in syms if is_usable_tool(sym))

In [ ]:
Markdown(mk_toollist([hi]))

- &`hi`: Say hi to `who`

In [ ]:
#| export
def import_gist(
    gist_id:str, # user/id or just id of gist to import as a module
    mod_name:str=None, # module name to create (taken from gist filename if not passed)
    add_global:bool=True, # add module to caller's globals?
    import_wildcard:bool=False, # import all exported symbols to caller's globals
    create_msg:bool=False # Add a message that lists usable tools
):
    "Import gist directly from string without saving to disk"
    fil = gist_file(gist_id)
    mod_name = mod_name or Path(fil['filename']).stem
    module = import_string(fil['content'], mod_name)
    glbs = currentframe().f_back.f_globals
    if add_global: glbs[mod_name] = module
    syms = getattr(module, '__all__', None)
    if syms is None: syms = [o for o in dir(module) if not o.startswith('_')]
    syms = [getattr(module, nm) for nm in syms]
    if import_wildcard:
        for sym in syms: glbs[sym.__name__] = sym
    if create_msg:
        pref = getattr(module, '__doc__', "Tools added to dialog:")
        add_msg(f"{pref}\n\n{mk_toollist(syms)}")
    return module

In [ ]:
import_gist(gistid)
importtest.testfoo

'testbar'

In [ ]:
import_gist.__doc__

'Import gist directly from string without saving to disk'

In [ ]:
import_gist(gistid, import_wildcard=True)
importtest.testfoo

'testbar'

In [ ]:
hi("Sarah")

'Hello Sarah'

In [ ]:
importtest.__all__

['hi', 'whoami']

## Tool info

This is how we get a superset of tools to include:

In [ ]:
#| export
def tool_info():
    cts='''Tools available from `dialoghelper`:

- &`curr_dialog`: Get the current dialog info.
- &`msg_idx`: Get absolute index of message in dialog.
- &`add_html`: Send HTML to the browser to be swapped into the DOM using hx-swap-oob.
- &`find_msg_id`: Get the current message id.
- &`find_msgs`: Find messages in current specific dialog that contain the given information.
  - (solveit can often get this id directly from its context, and will not need to use this if the required information is already available to it.)
- &`read_msg`: Get the message indexed in the current dialog.
- &`del_msg`: Delete a message from the dialog.
- &`add_msg`: Add/update a message to the queue to show after code execution completes.
- &`update_msg`: Update an existing message.'''
    add_msg(cts)

In [ ]:
for o in _all:
    s = globals()[o]
    if s.__name__[0]=='_' or not s.__doc__: continue
    print(f'- &`{s.__name__}`: {s.__doc__}')

- &`find_var`: Search for var in all frames of the call stack
- &`curr_dialog`: Get the current dialog info.
- &`find_msgs`: Find `list[dict]` of messages in current specific dialog that contain the given information. To refer to a message found later, use its `id` field.
- &`find_msg_id`: Get the message id by searching the call stack for __dialog_id.
- &`msg_idx`: Get absolute index of message in dialog.
- &`read_msg`: Get the `Message` object indexed in the current dialog.
- &`add_html`: Send HTML to the browser to be swapped into the DOM
- &`del_msg`: Delete a message from the dialog. Be sure to pass a `sid`, not a `mid`.
- &`add_msg`: Add/update a message to the queue to show after code execution completes.
- &`update_msg`: Update an existing message. Provide either `msg` OR field key/values to update.
    Use `content` param to update contents.
    Only include parameters to update--missing ones will be left unchanged.
- &`load_gist`: Retrieve a gist
- &`gist_file`: Get the first

## export -

In [ ]:
#|hide
from nbdev import nbdev_export
nbdev_export()